In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from threading import Thread
from timed_count import timed_count

from pprint import pprint

import datetime as dt
from tzlocal import get_localzone

import random
import time
import os

In [7]:
target = 'StackOverflow'

In [5]:
def scrap(data):
    allData = []
    start = dt.datetime.now(tz=get_localzone())
    fmt = "%m/%d/%y - %T %p"
    # Print starting output for app
    print('---'*20)
    print('---'*20)
    print(f'***** {target} started at {start.strftime(fmt)}')
    print()
    # looping data to scrap more details
    for item in data:
        # define target scrap based on question id
        url = 'https://stackoverflow.com/questions/' + str(item['question_id'])
        # make request to api
        req = requests.get(url)
        # parsing request with Beautifulsoup html parser
        soup = BeautifulSoup(req.text, 'html.parser')
        # defining variables
        question_id = str(item['question_id'])
        title = soup.find('a', {'class': 'question-hyperlink'}).text.strip() # type: ignore
        content = soup.find('div', {'class': 'js-post-body'}).text.strip()  # type: ignore
        post_tag = str(item['tags'])[1:-1]
        user_id = str(item['owner']['user_id'])
        question_stamp = soup.time.attrs # type: ignore
        # appending item to array
        allData.append({'question_id': question_id,'title': title, 'content': content, 'post_tag': post_tag, 'user_id': user_id, 'question_stamp': question_stamp['datetime']})
        
    # transform array to Pandas DataFrame
    check_out = pd.DataFrame(allData)
    # output path for storing data
    output_path = 'Var/StackOverFlow/' + target + '.csv'
    # storing data into csv format
    check_out.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    allData.clear()
    end = dt.datetime.now(tz=get_localzone())
    print(f"""
        Successfully inserted 20 {target} Questions into collection
        at {end.strftime(fmt)}.\n
        """)
    print(f'Time elapsed for {target}: {end-start}')
    print('---'*20)
    print('---'*20)
    print('\n')
    # Wait 1 to 5 seconds to start scraping next app
    # time.sleep(180)
        # with pd.ExcelWriter('Var/StackOverFlow/StackOverFlow.xlsx',
        #                     mode="a",
        #                     engine="openpyxl",
        #                     if_sheet_exists="overlay"
        #                     ) as writer:
        # check_out.to_excel(writer, sheet_name="datasheet", index=False)

In [ ]:
for count in timed_count(180):
    api = requests.get(
        'https://api.stackexchange.com/2.3/search/advanced?pagesize=20&order=desc&sort=activity&site=stackoverflow').json()
    data = api['items']
    scrap(data)


In [ ]:
# thread = Thread(target=scrap)
# thread.start()